<a href="https://colab.research.google.com/github/louistrue/learn-ifc-bfh25-D/blob/main/BFH-25-IFC-Dashboard-Starter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# IFC Dashboard Starter Template

A minimal template to get started building IFC data dashboards.

**What this does:**
- Load an IFC file from GitHub
- Extract element types and building storeys
- Create a simple 2-tab dashboard


## Step 1: Install packages


In [ ]:
%pip install -q ifcopenshell pandas plotly dash


## Step 2: Load IFC file from GitHub


In [ ]:
import urllib.request
import ifcopenshell

# Download IFC file from GitHub
github_url = 'https://raw.githubusercontent.com/louistrue/learn-ifc-bfh25-D/main/Modelle/BFH-25/02_BIMcollab_Example_ARC.ifc'
local_filename = '02_BIMcollab_Example_ARC.ifc'

print("📥 Downloading IFC file...")
urllib.request.urlretrieve(github_url, local_filename)
print("✅ Downloaded successfully!")

# Open IFC file
model = ifcopenshell.open(local_filename)
print(f"\n🏗️ Loaded: {model.schema}")
print(f"📊 Total elements: {len(model.by_type('IfcElement'))}")


## Step 3: Parse IFC data

Extract elements and their building storeys.


In [ ]:
import pandas as pd

# Helper function to get storey name
def get_storey_name(storey):
    if storey is None:
        return 'Not assigned'
    return storey.LongName or storey.Name or f"Storey {storey.id()}"

# Map elements to storeys
element_data = []

for element in model.by_type('IfcElement'):
    element_type = element.is_a()
    element_id = element.GlobalId
    
    # Find storey
    storey = 'Not assigned'
    for rel in model.by_type('IfcRelContainedInSpatialStructure'):
        if element in rel.RelatedElements:
            if rel.RelatingStructure and rel.RelatingStructure.is_a('IfcBuildingStorey'):
                storey = get_storey_name(rel.RelatingStructure)
                break
    
    element_data.append({
        'ElementId': element_id,
        'ElementType': element_type,
        'Storey': storey
    })

# Create DataFrame
df = pd.DataFrame(element_data)

print(f"✅ Parsed {len(df)} element records")
print(f"🏢 Found {df['Storey'].nunique()} storeys")
print(f"🔧 Found {df['ElementType'].nunique()} element types")

df.head()


## Step 4: Create aggregations for visualization


In [ ]:
# Count elements per storey
storey_counts = (
    df.groupby('Storey')['ElementId']
    .nunique()
    .reset_index(name='Count')
    .sort_values('Count', ascending=False)
)

# Count element types
type_counts = (
    df.groupby('ElementType')['ElementId']
    .nunique()
    .reset_index(name='Count')
    .sort_values('Count', ascending=False)
)

print("📊 Data ready for visualization!")
storey_counts.head()


## Step 5: Build a simple dashboard


In [ ]:
from dash import Dash, dcc, html
import plotly.express as px

# Create Dash app
app = Dash(__name__)

# Create charts
storey_chart = px.bar(
    storey_counts,
    x='Storey',
    y='Count',
    title='Elements per Storey',
    text_auto='.0f'
)
storey_chart.update_layout(xaxis_title='Building Storey', yaxis_title='Number of Elements')

type_chart = px.bar(
    type_counts.head(10),
    x='ElementType',
    y='Count',
    title='Top 10 Element Types',
    text_auto='.0f'
)
type_chart.update_layout(xaxis_title='Element Type', yaxis_title='Count', xaxis_tickangle=-45)

# Dashboard layout
app.layout = html.Div([
    html.H1('IFC Building Analysis'),
    html.P('A simple dashboard showing building elements and their distribution.'),
    
    dcc.Tabs([
        dcc.Tab(label='By Storey', children=[
            html.H3('How many elements are on each storey?'),
            dcc.Graph(figure=storey_chart)
        ]),
        
        dcc.Tab(label='By Type', children=[
            html.H3('What types of elements are in the building?'),
            dcc.Graph(figure=type_chart)
        ])
    ])
])

# Run the dashboard
print("🚀 Starting dashboard...")
app.run(jupyter_mode='inline', height=700, port=8050)


## Next Steps

**Extend this template:**
- Add more tabs for different analyses
- Include materials data
- Add filters with dropdowns
- Try different chart types

**Try other files:**
- Change the `github_url` to load different IFC files
- Upload your own IFC files using file upload widgets

**Learn more:**
- [IFCOpenShell Documentation](https://docs.ifcopenshell.org/)
- [Plotly Dash Documentation](https://dash.plotly.com/)
- [Full example: BFH-25-Tabbed-Dashboard.ipynb](https://github.com/louistrue/learn-ifc-bfh25-D/blob/main/BFH-25-Tabbed-Dashboard.ipynb)
